In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import json
import re
import os
import sys
sys.path.append('..')
from utilities import logger
from utilities import configuration


In [4]:
config = configuration.get_config()
for key in config:
    print(f'{key:40} (type: {type(config[key])})')


logging = logger.init_logger(config['system_log'])
logging.debug('Logger has started ont notebook 05. Analysing features.ipynb ...')

system_log                               (type: <class 'str'>)
json_file                                (type: <class 'str'>)
unified_merged_file_cz                   (type: <class 'str'>)
unified_merged_file_noncz                (type: <class 'str'>)
unified_merged_file                      (type: <class 'str'>)
data_path                                (type: <class 'str'>)
cz_files                                 (type: <class 'list'>)
noncz_files                              (type: <class 'list'>)
2023-09-28 11:57:03,294 - root - DEBUG - Logger has started ont notebook 05. Analysing features.ipynb ...


In [5]:
df = pd.read_csv(config['unified_merged_file'])
df.shape

/var/folders/d0/9279s72s163gx0_12qbmr9440000gn/T/ipykernel_4072/852065790.py:1: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(config['unified_merged_file'])


(618697, 31)

AssertionError: <class 'str'>

In [12]:
re.findall('([0-9\.][0-9\.]*)',df['Coded HCN'].iloc[256006])

['4988519.0', '170805.0', '170805.0']

In [23]:
for ix, hcn_code in enumerate(df['Coded HCN']):
    if isinstance(hcn_code, str):
        if '[' in hcn_code:
            if '[]'!=hcn_code:
                hcn_code=re.findall('([0-9\.][0-9\.]*)',df['Coded HCN'].iloc[256006])
                assert df['CZ status'].iloc[ix]=='Non-cz'
                
hcn_code



'2290324.0'

In [20]:

df['CZ status'].iloc[ix]

'Non-cz'

In [18]:
df.iloc[256006,:]

Institution Number:                                                        43
Coded HCN                                     [4988519.0, 170805.0, 170805.0]
Admit Date:                                               2014-12-22 00:00:00
Disch Date:                                               2015-01-01 00:00:00
Readmission Code:                                                           9
Patient Age:                                                               80
Gender:                                                                Female
MRDx                                           I500  Congestive heart failure
Postal Code:                                                           B2G1Y1
Diagnosis:                                          ['I500', 'E1152', 'J449']
Diagnosis Long Text         ['Congestive heart failure', 'Type 2 diabetes ...
Diagnosis Type                                                ['M', '3', '3']
Intervention Code                                               

In [76]:
from collections import defaultdict
import math
patient2record = defaultdict(list)
removed=set()
for ix in range(df.shape[0]):
    hcn_code = df['Coded HCN'].iloc[ix]
    if hcn_code != '[]' and hcn_code!='**':
        if '[' in hcn_code:
            hcn_code = re.findall('([0-9\.][0-9\.]*)', hcn_code)
            # 
            hcn_code = list(map(float, hcn_code))
            assert all([math.floor(number)==number for number in hcn_code])
            hcn_code = (set(map(math.floor, hcn_code)))
            if hcn_code!={170805}:
                hcn_code = list(filter(lambda numero: numero!=170805, hcn_code))
                assert len(hcn_code)==1, df['Coded HCN'].iloc[ix]
                hcn_code = hcn_code[0]

                patient2record[hcn_code].append(ix)
            
        else:
            hcn_code = float(hcn_code)
            assert hcn_code==math.floor(hcn_code)
            hcn_code = math.floor(hcn_code)
            patient2record[hcn_code].append(ix)
    else:
        removed.add(hcn_code)
removed

{'[]'}

In [75]:
record_count=0
for patient in patient2record:
    record_count+=len(patient2record[patient])
print((record_count))
print(len(patient2record))

617980
298371


In [64]:
hcn_code = df['Coded HCN'].iloc[256006]
hcn_code = re.findall('([0-9\.][0-9\.]*)', hcn_code)
hcn_code = map(float, hcn_code)
hcn_code = tuple(sorted(set(map(math.floor, hcn_code))))
assert len(list(filter(lambda numero: numero!=170805, hcn_code)))==1, str(ix)

list(hcn_code)

[170805, 4988519]

In [45]:
tuple_ = (170805, 4988519)


In [47]:
list(filter(lambda x: x!=170805, tuple_))

[4988519]

In [77]:
df.shape

(618697, 31)

In [81]:
df.columns[-5:]

Index(['Discharge Nurse Unit', 'is ALC Patient', 'Disch Date (year-month):',
       'Total Days in Hospital', 'CZ status'],
      dtype='object')